In [ ]:
# Chosen dataset
# https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [748 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.2 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bioni

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-14 05:07:29--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-05-14 05:07:29 (5.95 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract Data

In [4]:
from pyspark import SparkFiles
# Load in Amazon Reviews Books from S3 into a DataFrame
# url = "https://<bucket name>.s3.amazonaws.com/employee.csv"
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Books_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   25933450| RJOVP071AVAJO|0439873800|      84656342|There Was an Old ...|           Books|          5|            0|          0|   N|                Y|          Five Stars|I love it and so ...| 2015-08-31|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend|           Books|          5|    

In [14]:
# Create view to make spark sql available to use
df.createOrReplaceTempView("books")

In [15]:
# Change the type of review_date to date type
from pyspark.sql.functions import to_date

df = df.withColumn("review_date", to_date(df["review_date"], "yyyy-MM-dd"))
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [16]:
# Count the number of records in the dataset
df.count()

10319090

# Transform Data

## customers_table
customer_id (PK) (integer)<br>
customer_count (integer)<br>

In [17]:
# Create customers_table
customers_table = spark.sql("select customer_id, count(customer_id) as customer_count from books group by customer_id")
customers_table.show()
customers_table.printSchema()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16747249|             4|
|     108460|             1|
|   11720077|            31|
|     437936|             1|
|     128131|             1|
|   39647402|             1|
|   13730111|             8|
|   43783459|             1|
|   12493167|           111|
|   40325486|            92|
|   44618558|             4|
|   17956020|             1|
|   10093406|             3|
|   40430762|             1|
|     134924|             1|
|   52427934|             1|
|   12879980|            15|
|   27680937|             7|
|   16186289|             1|
|   20326078|             1|
+-----------+--------------+
only showing top 20 rows

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [9]:
customers_table = customers_table.withColumn("customer_count", customers_table["customer_count"].cast("integer"))
customers_table.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



## products_table
product_id (PK) (string)<br>
product_title (string)<br>

In [18]:
# Create products_table
products_table = spark.sql("select product_id, product_title from books")
products_table.show()
products_table.printSchema()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0439873800|There Was an Old ...|
|1623953553|      I Saw a Friend|
|142151981X|Black Lagoon, Vol. 6|
|014241543X|           If I Stay|
|1604600527|Stars 'N Strips F...|
|0399170863|            The Liar|
|1517007240|Devil in the Deta...|
|0671728725|Knowing When to S...|
|1111349533|The American Pageant|
|812211637X|Punjabi C.L. Bibl...|
|0679413359|The Aeneid (Every...|
|0983990662|The YMCA of Middl...|
|156852062X|Memoirs By Harry ...|
|B0007E4WJ4|Arms of our fight...|
|0664254969|Presbyterian Cree...|
|1570913722|Hello Ocean/Hola mar|
|0345313860|The Vampire Lesta...|
|0670025593|The Book of Life ...|
|161893158X|Disney Descendant...|
|007179512X|CompTIA A+ Certif...|
+----------+--------------------+
only showing top 20 rows

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



## review_id_table
review_id (PK) (string)<br>
customer_id (integer)<br>
product_id (string)<br>
product_parent (integer)<br>
review_date (date)<br>

In [19]:
# Create review_id_table
review_id_table = spark.sql("select review_id, customer_id, product_id, product_parent, review_date from books")
review_id_table.show()
review_id_table.printSchema()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RJOVP071AVAJO|   25933450|0439873800|      84656342| 2015-08-31|
|R1ORGBETCDW3AI|    1801372|1623953553|     729938122| 2015-08-31|
| R7TNRFQAOUTX5|    5782091|142151981X|     678139048| 2015-08-31|
|R2GANXKDIFZ6OI|   32715830|014241543X|     712432151| 2015-08-31|
|R2NYB6C3R8LVN6|   14005703|1604600527|     800572372| 2015-08-31|
|R13U5PBJI1H94K|   36205738|0399170863|     559876774| 2015-08-31|
|R1H8UVH990F8VE|   44121167|1517007240|     299984591| 2015-08-31|
|R2MC0N30WZMRQ5|   16519255|0671728725|     821650353| 2015-08-31|
| R2NO2HXK16Y4J|   49361350|1111349533|     419457767| 2015-08-31|
|R245YIAVJK82ZL|   11423253|812211637X|     285887177| 2015-08-31|
| RFBMLBWQOZ1UM|   37536260|0679413359|      58983679| 2015-08-31|
| RHE1FEWV3KTQH|   23614950|0983990662|     222024699| 2015-08

## vine_table
review_id (PK) (string)<br>
star_rating (integer)<br>
helpful_votes (integer)<br>
total_votes (integer)<br>
vine (string)<br>

In [20]:
# Create vine_table
vine_table = spark.sql("select review_id, star_rating, helpful_votes, total_votes, vine from books")
vine_table.show()
vine_table.printSchema()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RJOVP071AVAJO|          5|            0|          0|   N|
|R1ORGBETCDW3AI|          5|            0|          0|   N|
| R7TNRFQAOUTX5|          5|            0|          0|   N|
|R2GANXKDIFZ6OI|          5|            0|          0|   N|
|R2NYB6C3R8LVN6|          5|            2|          2|   N|
|R13U5PBJI1H94K|          2|            1|          1|   N|
|R1H8UVH990F8VE|          5|            2|          2|   N|
|R2MC0N30WZMRQ5|          5|            0|          0|   N|
| R2NO2HXK16Y4J|          5|            0|          0|   N|
|R245YIAVJK82ZL|          5|            0|          0|   N|
| RFBMLBWQOZ1UM|          5|            0|          0|   N|
| RHE1FEWV3KTQH|          5|            0|          0|   N|
|R2BB9U7PHO7ETO|          5|            0|          1|   N|
|R28XJJP754GYXO|          4|            

# Load Data

In [21]:
# Configure settings for RDS
mode = "append"
# jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
jdbc_url="jdbc:postgresql://database-bigdata.czywd26sbfeu.us-west-2.rds.amazonaws.com:5432/datasetlvl2"

config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [22]:
# Write DataFrame to customers_table table in RDS
customers_table.write.jdbc(url = jdbc_url, table = 'customers', mode = mode, properties = config)

In [24]:
# Write DataFrame to products_table table in RDS
products_table.write.jdbc(url = jdbc_url, table = 'products', mode = "overwrite", properties = config)

In [25]:
# Write DataFrame to review_id_table table in RDS
review_id_table.write.jdbc(url = jdbc_url, table = 'review_id_table', mode = mode, properties = config)

In [27]:
# Write DataFrame to vine_table table in RDS
vine_table.write.jdbc(url = jdbc_url, table = 'vine_table', mode = "overwrite", properties = config)